In [1]:
import sqlite3
import networkx as nx
from collections import deque

In [48]:
dict_stations={}
class loop:
    global dict_stations
    def __init__(self,number):
        self.loop_no=number
        self.direction='none'
        self.loop_type='ml'
        self.station_name='none'
        self.allowed = 'all'
        self.loop_vel =0
        self.uplinks=[]
        self.uplink_len=0
        self.up_priority=1
        self.up_crossover='0'
        self.dnlinks=[]
        self.dnlink_len=0
        self.dn_priority=1
        self.dn_crossover='0'
        self.speed_restriction=150
        self.startmilep=0
        self.endmilep=0
        self.lines=0


    def add_loop(self,direction,loop_type,station_name,allowed,loop_vel,uplinks,uplink_len,up_priority,
                 up_crossover,dnlinks,dnlink_len,dn_priority,dn_crossover,speed_restriction,startmilep,endmilep,lines):
        
        self.direction=direction
        self.loop_type=loop_type
        self.station_name=station_name
        self.allowed = allowed
        self.loop_vel =loop_vel
        self.uplinks.extend(uplinks.split(","))
        self.uplink_len=uplink_len
        self.up_priority=up_priority
        self.up_crossover=up_crossover
        self.dnlinks.extend(dnlinks.split(","))
        self.dnlink_len=dnlink_len
        self.dn_priority=dn_priority
        self.dn_crossover=dn_crossover
        self.speed_restriction=speed_restriction
        self.startmilep=startmilep
        self.endmilep=endmilep
        self.lines=lines
        dict_stations[self.station_name]=self.loop_no
 
        
    def get_station(self):
        return self.station_name
    
    def get_loop_no(self):
        return self.loop_no
    
    def get_direction(self):
        return self.direction
    
    def get_loop_type(self):
        return self.loop_type
    
    def get_uplinks(self):
        return self.uplinks
    
    def get_dnlinks(self):
        return self.dnlinks
    
    def get_speed_restrictions(self):
        return self.speed_restriction
    
    def get_startmilep(self):
        return self.startmilep
    
    def get_endmilep(self):
        return self.endmilep

        
    

In [49]:
class block:
    def __init__(self,number):
        self.block_no=number
        self.direction='none'
        self.startmilep=0
        self.endmilep=0
        self.block_velocity=0
        self.uplinks=[]
        self.uplink_len=0
        self.up_priority=1
        self.up_crossover='0'
        self.dnlinks=[]
        self.dnlink_len=0
        self.dn_priority=1
        self.dn_crossover='0'
        self.speed_restriction=150
        self.startmileposts=0
        self.endmileposts=0
        self.bwt=0
        self.signal = 0  #0 is green 
        
    def add_block(self,direction,startmilep,endmilep,block_velocity,uplinks,uplink_len,up_priority,
                 up_crossover,dnlinks,dnlink_len,dn_priority,dn_crossover,speed_restriction,startmileposts,endmileposts,bwt):
        
        self.direction=direction
        self.startmilep=startmilep
        self.endmilep=endmilep
        self.block_velocity=block_velocity
        self.uplinks.extend(uplinks.split(","))
        self.uplink_len=uplink_len
        self.up_priority=up_priority
        self.up_crossover=up_crossover
        self.dnlinks.extend(dnlinks.split(","))
        self.dnlink_len=dnlink_len
        self.dn_priority=dn_priority
        self.dn_crossover=dn_crossover
        self.speed_restriction=speed_restriction
        self.startmileposts=startmileposts
        self.endmileposts=endmileposts
        self.bwt=bwt
        
    
    def get_block_no(self):
        return self.block_no
    
    def get_direction(self):
        return self.direction
    
    def get_uplinks(self):
        return self.uplinks
    
    def get_dnlinks(self):
        return self.dnlinks
    
    def get_speed_restrictions(self):
        return self.speed_restriction
    
    def get_startmilep(self):
        return self.startmilep
    
    def get_endmilep(self):
        return self.endmilep

        
    

In [97]:
class train:
    def __init__(self,number):
        self.train_no=number
        self.direction='none'
        self.linked_train=''
        self.linking_time=''
        self.start_time=0
        self.length=0
        self.accl=0
        self.decl=0
        self.priority=0
        self.max_speed=0
        self.startloop='0'
        self.endloop='0'
        self.dilemma_breaker=''
        self.num_halts=0
        self.halts_min={}
        self.block=[]
        self.arr_timings=[]
        self.dep_timings=[]
        
    def add_train(self,direction,linked_train,linking_time,start_time,length,accl,decl,priority,max_speed,dilemma_breaker,
                  startloop,endloop,num_halts,halts_min):
        
        self.direction=direction
        self.linked_train=linked_train
        self.linking_time=linking_time
        self.start_time=start_time
        self.length=length
        self.accl=accl
        self.decl=decl
        self.priority=priority
        self.max_speed=max_speed
        self.startloop=startloop
        self.endloop=endloop
        self.dilemma_breaker=dilemma_breaker
        self.num_halts=num_halts
        halts=halts_min.split(",")[0::2]
        timings =[ float(i) for i in halts_min.split(",")[1::2]]
        self.halts_min = dict(zip(halts,timings))
        
    
    def get_train_no(self):
        return self.train_no
    
    def get_startloop(self):
        return self.startloop
    
    def get_endloop(self):
        return self.endloop
    
    def get_direction(self):
        return self.direction
   
    def get_halts(self):
        return self.halts_min
    
    def get_start_time(self):
        return self.start_time
    
    def get_accl(self):
        return self.accl
    def get_decl(self):
        return self.decl
    
    def get_max_speed(self):
        return self.max_speed
    
    def get_halt_mins(self):
        return self.halts_min

In [98]:
conn = sqlite3.connect("ZBTT.db")
c = conn.cursor()
get_loop_query="""select * from Loop_Combined"""
get_blocks_query="""select * from Block_Combined"""
get_trains_query="""select * from Unschedule_Combined"""
c.execute(get_loop_query)
loops=c.fetchall()
c.execute(get_blocks_query)
blocks=c.fetchall()
c.execute(get_trains_query)
trains=c.fetchall()

In [99]:
loop_numbers = {}
for i in loops:
    loop_numbers[str(i[0])] = loop(i[0]) 
    loop_numbers[str(i[0])].add_loop(*i[1:])

In [100]:
loop_numbers['20180001'].get_startmilep()

''

In [101]:
block_numbers = {}
for i in blocks:
    block_numbers[str(i[0])] = block(i[0])
    i=[j.strip('"') if type(j)==str else j for j in i ]
    block_numbers[str(i[0])].add_block(*i[1:])

In [102]:
block_numbers['20180010'].get_startmilep()

138.94000000000028

In [103]:
train_numbers = {}
for i in trains:
    train_numbers[str(i[0])] = train(i[0])
    i=[j.strip('"') if type(j)==str else j for j in i ]
    train_numbers[str(i[0])].add_train(*i[1:])

In [104]:
last_loop = '20180001'
next_links=[]
first_loop='19010001'
while first_loop!=last_loop:
    if first_loop[-1]=='1':
        if (loop_numbers[first_loop].get_direction() =='up' ):
                uplinks=loop_numbers[first_loop].get_uplinks()
                next_links.append(uplinks[0])
                first_loop=uplinks[0]
    else:
        if (block_numbers[first_loop].get_direction() =='up' ):
                uplinks=block_numbers[first_loop].get_uplinks()
                next_links.append(uplinks[0])
                first_loop=uplinks[0]

In [105]:
G_up = nx.DiGraph()
G_down = nx.DiGraph()
# adding all loops and blocks as nodes:

for node in block_numbers:
    if(block_numbers[node].get_direction() == "up"):
        G_up.add_node(node, direction="up", nature="block")
    elif(block_numbers[node].get_direction() == "down"):
        G_down.add_node(node, direction="down", nature="block")
    else:
        G_up.add_node(node, direction="common", nature="block")
        G_down.add_node(node, direction="common", nature="block")

up_blocks_nodes_list = list(G_up.nodes(data=True))
down_blocks_nodes_list = list(G_down.nodes(data=True))

for node in loop_numbers:
    if(loop_numbers[node].get_direction() == "up"):
        G_up.add_node(node, direction="up", station=loop_numbers[node].get_station(), nature="loop")
    elif(loop_numbers[node].get_direction() == "down"):
        G_down.add_node(node, direction="down", station=loop_numbers[node].get_station(), nature="loop")
    else:
        G_up.add_node(node, direction="common", station=loop_numbers[node].get_station(), nature="loop")
        G_down.add_node(node, direction="common", station=loop_numbers[node].get_station(), nature="loop")


In [106]:
for node in block_numbers:
    if(block_numbers[node].get_direction() == 'up'):
        for nNode in block_numbers[node].get_uplinks():
            G_up.add_edge(node, nNode, direction='up')
    elif(block_numbers[node].get_direction() == 'down'):
        for nNode in block_numbers[node].get_dnlinks():
            G_down.add_edge(node, nNode, direction='down')
    else:
        for nNode in block_numbers[node].get_uplinks():
            G_up.add_edge(node, nNode, direction='up')
        for nNode in block_numbers[node].get_dnlinks():
            G_down.add_edge(node, nNode, direction='down')

for node in loop_numbers:
    if(loop_numbers[node].get_direction() == 'up' and len(loop_numbers[node].get_uplinks()) != 0):
        for nNode in loop_numbers[node].get_uplinks():
            G_up.add_edge(node, nNode, direction='up')
    elif(loop_numbers[node].get_direction() == 'down' and len(loop_numbers[node].get_dnlinks()) != 0):
        for nNode in loop_numbers[node].get_dnlinks():
            G_down.add_edge(node, nNode, direction='down')
    else:
        if(len(loop_numbers[node].get_uplinks()) != 0):
            for nNode in loop_numbers[node].get_uplinks():
                G_up.add_edge(node, nNode, direction='up')
        if(len(loop_numbers[node].get_dnlinks()) != 0):
            for nNode in loop_numbers[node].get_dnlinks():
                G_down.add_edge(node, nNode, direction='down')

In [246]:
def print_traversal(loop_array,time_array):
    print(len(loop_array))
    print(len(time_array))
    for i in range(len(loop_array)):
        if loop_array[i][-1]=='0':
            print(loop_array[i]," ",time_array[2*i]," ",time_array[2*i+1])
        elif loop_array[i][-1]=='1':
            print(loop_array[i]," ",time_array[2*i]," ",time_array[2*i+1],"  ",loop_numbers[loop_array[i]].get_station())


    
def get_time(start_velocity,accl,distance):
    
    if start_velocity==0:
        traversal_time = np.sqrt((2*distance)/accl) 
        end_vel = start_velocity + accl*traversal_time
        if end_vel>max_speed:
            end_vel=max_speed
    else:
        end_vel = start_velocity**2 + 2*accl*distance
    
        if end_vel>max_speed:
            end_vel=max_speed
        traversal_time = (end_vel - start_velocity) / accl
        if traversal_time == 0 :
            traversal_time = distance / end_vel
    return end_vel,traversal_time



def traverse_block(graph,train_number,start_time,start,end,accl,decl,max_speed,time_array,loop_array,
                   start_velocity,halt,duration):
    
    if str(start)[-1]=='1':
        if str(start)[0:4]==str(end)[0:4]:
            time_array.append(start_time)
            time_array.append(0)
            
            time_array = [i/60 for i in time_array]
            print_traversal(loop_array,time_array)
            return True
   
        
    if str(start)[-1]=='1':      
        distance = 0.75*1000
    elif str(start)[-1]=='0':
        distance = (float(block_numbers[start].get_endmilep()) - float(block_numbers[start].get_startmilep()))*1000
    
    end_vel,traversal_time = get_time(start_velocity,accl,distance)
    
    if halt==1:
        traversal_time = traversal_time + duration
    loop_end_time = start_time + traversal_time
    
    next_loop = graph[start]
    next_loop = [i for i in next_loop]
    next_loop=next_loop[0]
    if next_loop[-1]=='1':
        station = loop_numbers[next_loop].get_station()
    else:
        station = None
    if station in train_numbers[train].get_halts():
        halt_mins = train_numbers[train].get_halts()
        duration = halt_mins[station]*60
        accl=decl
        end_vel = 0
        #print(duration)
        halt=1
    else:
        halt=0
    loop_array.append(next_loop)
    time_array.append(loop_end_time)
    time_array.append(loop_end_time)
    traverse_block(graph,train_number,loop_end_time,next_loop,end,accl,decl,max_speed,time_array,loop_array,end_vel,halt,duration)

In [247]:
print(train_numbers[train].get_halts())
print(loop_numbers['90801001'].get_station())

{'KYN': 3.0, 'MHLC': 3.15, 'KAD': 3.15, 'SVJR': 4.84, 'PUNE': 5.0, 'PAA': 2.07, 'DDAP': 2.07, 'KWV': 2.0, 'SUR': 5.0, 'DUD': 2.0, 'GUR': 2.0, 'KLBG': 11.22, 'HQR': 8.22, 'MR': 8.22, 'SDB': 10.22, 'WADI': 5.0, 'YG': 1.0, 'KSN': 2.0, 'RC': 2.0, 'MALM': 1.0, 'AD': 1.0, 'GTL': 5.0, 'NKDO': 7.06, 'PLU': 7.06, 'GY': 8.06}
GY


In [249]:
import numpy as np
train='11011013'
if train_numbers[train].get_direction()=='up':
    graph=G_up
else:
    graph=G_down
    
start_time = float(train_numbers[train].get_start_time()) * 60
start_loop = str(train_numbers[train].get_startloop())
end_loop = str(train_numbers[train].get_endloop())
accl=train_numbers[train].get_accl()
decl=train_numbers[train].get_decl()
max_speed=train_numbers[train].get_max_speed()*(10/36)
traverse_block(graph,train,start_time,start_loop,end_loop,accl,decl,max_speed,[start_time],[start_loop],0,0,0)


239
479
91981001   1381.0   1382.8257418583505    TNA
91971010   1382.8257418583505   1385.0
91971001   1385.0   1385.4166666666667    DIVA
91961010   1385.4166666666667   1388.1111111111109
91961001   1388.1111111111109   1388.5277777777776    DI
91951010   1388.5277777777776   1390.9722222222222
91951001   1390.9722222222222   1395.263216670958    KYN
91941010   1395.263216670958   1395.9722222222222
91941001   1395.9722222222222   1396.388888888889    VLDI
91931010   1396.388888888889   1396.877777777778
91931001   1396.877777777778   1397.2944444444445    ULNR
91921010   1397.2944444444445   1398.3277777777778
91921001   1398.3277777777778   1398.7444444444445    ABH
91911010   1398.7444444444445   1402.4555555555557
91911001   1402.4555555555557   1402.8722222222225    BUD
91901010   1402.8722222222225   1408.838888888889
91901001   1408.838888888889   1409.2555555555557    VGI
91891010   1409.2555555555557   1410.8166666666668
91891001   1410.8166666666668   1411.2333333333336   

91031001   1788.0711111111127   1791.3621055598485    KSN
91021010   1791.3621055598485   1792.0711111111127
91021001   1792.0711111111127   1792.4877777777795    YDLP
91011010   1792.4877777777795   1795.432222222224
91011001   1795.432222222224   1795.8488888888905    CK
91001010   1795.8488888888905   1799.532222222224
91001001   1799.532222222224   1799.9488888888907    YS
90991010   1799.9488888888907   1804.5655555555572
90991001   1804.5655555555572   1807.856550004293    RC
90981010   1807.856550004293   1808.5655555555572
90981001   1808.5655555555572   1808.982222222224    MRC
90971010   1808.982222222224   1812.5766666666684
90971001   1812.5766666666684   1812.9933333333352    MTU
90961010   1812.9933333333352   1817.3544444444462
90961001   1817.3544444444462   1817.771111111113    HPH
90951010   1817.771111111113   1818.771111111113
90951001   1818.771111111113   1821.0621055598488    MALM
90941010   1821.0621055598488   1821.771111111113
90941001   1821.771111111113   18

In [35]:
def check_path(path,halts):

    for i in halts:
        if i not in path:
            print("in false")
            return False
    return True

def dfs(graph, src, dest, discovered, path,dir,halts):
       
    discovered.add(src)
    path.append(src)
    if src == dest:
        return True        
    else: 
        x=graph[src]                
    #print(x)
    for i in x:
        if i not in discovered:
            if dfs(graph, i, dest, discovered, path,dir,halts):
                return True
                if check_path(path,halts): 
                    return True
    path.pop()
    return False

NameError: name 'loop_array' is not defined

In [18]:
def search(graph, origin, destination, search_type):
  
    path_list = [[origin]]
 
    while path_list:       
         
        if search_type == 'bfs':
            pop_index = 0
        if search_type == 'dfs':
            pop_index = -1
        path = path_list.pop(pop_index)

        last_node = path[-1]
        if last_node == destination:  
            return path

        else:
            for node in graph[last_node]:
                if node not in path:
              
                    new_path = path + [node]  
                 
                    path_list.append(new_path)  

    print('No path exists between %s and %s' %(origin, destination))


In [25]:
discovered=set()
path=deque()
train='11011058'
if train_numbers[train].get_direction()=='up':
    graph=G_up
else:
    graph=G_down

halts=train_numbers[train].get_halts()
halts_loop = [str(dict_stations[i]) for i in halts]
search(graph,str(train_numbers[train].get_startloop()),str(train_numbers[train].get_endloop()),'dfs')


KeyboardInterrupt: 

In [16]:
discovered=set()
path=deque()
train='11012839'
if train_numbers[train].get_direction()=='up':
    graph=G_up
else:
    graph=G_down

halts=train_numbers[train].get_halts()
halts_loop = [str(dict_stations[i]) for i in halts]
dfs(graph,str(train_numbers[train].get_startloop()),str(train_numbers[train].get_endloop()),discovered,path,train_numbers[train].get_direction(),halts_loop)
path

deque(['30010001',
       '30010010',
       '30020001',
       '30020010',
       '30030001',
       '30030010',
       '30040001',
       '30040010',
       '30050001',
       '30050010',
       '30060001',
       '30060010',
       '30070001',
       '30070010',
       '30080001',
       '30080010',
       '30090001',
       '30090010',
       '30100001',
       '30100010',
       '30110001',
       '30110010',
       '30120001',
       '30120010',
       '30130001',
       '30130010',
       '30140001',
       '30140010',
       '30150001',
       '30150010',
       '30160001',
       '30160010',
       '30170001',
       '30170010',
       '30180001',
       '30180010',
       '30190001',
       '30190010',
       '30200001',
       '30200010',
       '30210001',
       '30210010',
       '30220001',
       '30220010',
       '30230001',
       '30230010',
       '30240001',
       '30240010',
       '30250001',
       '30250010',
       '30260001',
       '30260010',
       '3027

In [69]:
def printAllPathsUtil(graph, u, d, visited, path): 
    visited[u[0:7]]= True
    path.append(u) 
    if u == d: 
        print(path) 
    else: 
        for i in graph[u]: 
            if visited[i[0:7]]== False: 
                printAllPathsUtil(graph,i, d, visited, path) 
    path.pop() 
    visited[u[0:7]]= False

In [70]:
visited=[False]*G_up.number_of_nodes()
loops = list(G_up)
loops = [i[0:7] for i in loops]
visited = dict(zip(loops,visited))
path=[]
printAllPathsUtil(graph,str(train_numbers[train].get_startloop()),str(train_numbers[train].get_endloop()),visited,path)



KeyboardInterrupt: 

In [22]:
for i in G_up['330350010']:
    print(i)


30360001
30360101
30360102


In [123]:

# Python program to print all paths from a source to destination. 
   
from collections import defaultdict 
   
# This class represents a directed graph  
# using adjacency list representation 
class Graph: 
   
    def __init__(self, vertices): 
        # No. of vertices 
        self.V = vertices  
          
        # default dictionary to store graph 
        self.graph = defaultdict(list)  
   
    # function to add an edge to graph 
    def addEdge(self, u, v): 
        self.graph[u].append(v) 
   
    '''A recursive function to print all paths from 'u' to 'd'. 
    visited[] keeps track of vertices in current path. 
    path[] stores actual vertices and path_index is current 
    index in path[]'''
    def printAllPathsUtil(self, u, d, visited, path): 
  
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
  
        # If current vertex is same as destination, then print 
        # current path[] 
        if u == d: 
            print(path )
        else: 
            # If current vertex is not destination 
            # Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]== False: 
                    self.printAllPathsUtil(i, d, visited, path) 
                      
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop() 
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self, s, d): 
  
        # Mark all the vertices as not visited 
        visited =[False]*(self.V) 
  
        # Create an array to store paths 
        path = [] 
  
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d, visited, path) 
   
   
   
# Create a graph given in the above diagram 
g = Graph(4) 
g.addEdge(0, 1) 
g.addEdge(0, 2) 
g.addEdge(0, 3) 
g.addEdge(2, 0) 
g.addEdge(2, 1) 
g.addEdge(1, 3) 
   
s = 2 ; d = 3
print ("Following are all different paths from % d to % d :" %(s, d)) 
g.printAllPaths(s, d) 
# This code is contributed by Neelam Yadav 


Following are all different paths from  2 to  3 :
[2, 0, 1, 3]
[2, 0, 3]
[2, 1, 3]
